In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
plt.rcParams["font.family"] = "Times New Roman"

from utils import get_hdf_keys
from train import training
from evaluation import evaluate, get_run_results
import warnings
import pickle
warnings.filterwarnings('ignore')
# Set seed
np.random.seed(42)

In [2]:
# Set working directory
import wandb
wandb.login()

os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"
dir_path = os.path.join(os.getcwd(), 'data', 'clean_data')
model_dir = os.path.join(os.getcwd(), 'models')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


Current working directory: c:\Users\nik\Desktop\Berkeley_Projects\WattCast


In [3]:
locations_dict, resolutions_dict = get_hdf_keys(dir_path)

In [4]:
locations_dict

{'2_town.h5': ['GLENDOVEER-13596',
  'GLENDOVEER-13597',
  'GLENDOVEER-13598',
  'GLENDOVEER-13599',
  'GLENDOVEER-CLIFFGATE',
  'GLENDOVEER-NORTHEAST',
  'KELLY',
  'LENTS-13101',
  'LENTS-HAPPY',
  'LENTS-MT',
  'LENTS-NORTH',
  'MIDWAY-DIVISION',
  'MIDWAY-DOUGLAS',
  'MIDWAY-LYNCH',
  'MIDWAY-POWELLHURST',
  'RAMAPO-EMERALD',
  'RAMAPO-GILBERT',
  'RAMAPO-RAMAPO']}

### Loop through all locations and resolutions

In [ ]:
for scale, locations in locations_dict.items():

    scale = scale.split('.')[0]
    for location in locations:

        print(f"Training {location} at {scale} scale")
        config, models = training(scale, location, tuned_models=['xgb']) # loads existing models (from disk) if they exist, otherwise trains new models with optimial hyperparameters (from wandb) if they exist
        eval_dict = evaluate(config, models) # loads existing run results (from wandb) if they exist, otherwise runs a backtest for each model on the val and test set, and then formats it into the various horizons
        df_metrics = get_run_results(eval_dict, config) # calculates the error scores and produces plots, logging them to wandb if possible

        wandb.finish()

### Run for a single location

In [5]:
scale = '2_town'
location = 'GLENDOVEER-13596'

print(f"Training {location} at {scale} scale")

config_run = {
    'spatial_scale': scale,
    'temp_resolution': 60,
    'location': location,
    'tuned_models': ['tide'],
    'horizon_in_hours': 24,
    'lookback_in_hours': 24,
    'boxcox': True,
    'liklihood': None,
    'weather_available': True,
    'datetime_encodings': True,
    'heat_wave_binary': True,
}

config, models = training(config_run)

eval_dict = evaluate(config, models)

df_metrics = get_run_results(eval_dict, config)

wandb.finish()

Training GLENDOVEER-13596 at 2_town scale
Could not find a sweep for model tide and scale 2_town in project Wattcast_tuning.
Fetched sweep with name None for model tide


AttributeError: 'Config' object has no attribute 'temp_resolution'

## Debugging